In [1]:
import os
import email
import re
import mailparser
import random
import spacy
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tld import get_tld
from time import strftime, strptime, time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc, recall_score
from itertools import product
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

Set constants for directories, take a random sampling of the "ignore" email since there are so many (under sample)

In [2]:
IGNORE_DIR = '/Users/worshamn/Documents/emailProject/IgnoreFix'
INVESTIGATE_DIR = '/Users/worshamn/Documents/emailProject/InvestigateFix'
#https://stackoverflow.com/a/6482922
random.seed(2842)
ignore_sample_index = [ i for i in sorted(random.sample(range(len(os.listdir(IGNORE_DIR))), 400)) ]
ignore_sample = []
for i in ignore_sample_index:
    ignore_sample.append(os.listdir(IGNORE_DIR)[i])
input_dirs = {}
input_dirs[INVESTIGATE_DIR] = os.listdir(INVESTIGATE_DIR) 
input_dirs[IGNORE_DIR] = ignore_sample

In [3]:
ignore_sample

['[Phish Alert] FW- Barbara.jones shrink your belly, 1lb day.eml',
 '[Phish Alert] FW- 9 Signs You’re Successful—Even If It Doesn’t Feel Like It.eml',
 '[Phish Alert] FW- Watch us print human tissue-organs.eml',
 '[Phish Alert] FW- Share your thoughts, Sara.eml',
 '[Phish Alert] FW- [Spam] NEW! Azure This Month.eml',
 '[Phish Alert] FW- 5 Reasons Companies Love Surface Pro.eml',
 '[Phish Alert] FW- Wells Fargo Wants Your Input on the Custom Credit Offer! - Reminder.eml',
 '[Phish Alert] FW- [Spam] RE- Couple questions go-forward.eml',
 '[Phish Alert] FW- Quality Pet Food Delivered.eml',
 '[Phish Alert] FW- Get the most out of your lectures.eml',
 '[Phish Alert] FW- Imagine Speaking a New Language in Just 30 Days - Free Trial.eml',
 '[Phish Alert] FW- Now’s the Time to Act – Dedicated Servers at Excellent Prices.eml',
 '[Phish Alert] FW- Fw-  to Steve Lindell.eml',
 '[Phish Alert] FW- An invitation to support \uf020My Family Is In Need Of A Blessing\uf020.eml',
 '[Phish Alert] FW- My de

len(input_dirs)

In [4]:
len(input_dirs[INVESTIGATE_DIR])

384

In [5]:
len(input_dirs[IGNORE_DIR])

400

Build function to extract text and features

In [6]:
def get_email_text(file):
    d = {}
    raw_message = email.message_from_file(file)
    mail = mailparser.parse_from_string(raw_message.as_string())
    d['subject'] = mail.subject
    d['subject_len'] = len(d['subject'])
    if raw_message.is_multipart():
        d['is_mulitpart'] = 1
    else:
        d['is_multipart'] = 0
    d['body'] = mail.text_plain
    if len(d['body']) > 0:
        d['mail_text'] = d['subject'] + ' ' + d['body'][0]
        d['body_len'] = len(d['body'][0])
        if len(d['body']) > 1:
            soup_html = BeautifulSoup(d['body'][1],'lxml')
            d['links'] = soup_html.find_all('a')
            d['num_link'] = len(d['links'])
            links = []
            d['masq_link'] = []
            d['masq_link_tld'] = []
            d['num_email_link'] = 0
            for link in d['links']:
                link_text = link.get_text().rstrip('\n')
                a_link = link.get('href')
                links.append(a_link)
                if 'unsubscribe' in link_text.lower():
                    d['has_unsubscribe_link'] = 1
                if a_link:    
                    if re.search('mailto:',a_link):
                        d['num_email_link'] += 1
                if a_link != link_text and \
                    'http' in link_text.lower() and \
                    not 'alt="http' in link_text.lower():
                        d['masq_link'].append(link)
                        d['masq_link_tld'].append(
                            get_tld(
                                a_link,
                                fix_protocol=True, 
                                fail_silently=True
                            )
                        )
            d['num_uniq_link'] = len(set(links))
            if d['num_link'] > d['num_uniq_link']:
                d['has_repeatlink'] = 1
            else:
                d['has_repeatlink'] = 0
            if len(d['masq_link']) == 0:
                d['masq_link'] = ''
                d['masq_link_tld'] = ''
                d['has_masq_link'] = 0
            else:    
                d['has_masq_link'] = 1
                d['num_masq_link'] = len(d['masq_link'])
    else:
        d['mail_text'] = d['subject']
        d['body_len'] = len(d['body'])
    url_query = '((?:https?|ftp)://[^\s/$.?#]+\.[^\s>]+)'
    d['url'] = re.findall(url_query,d['mail_text'])
    email_query = '([\w.]+@[\w.]+\.[\w.]{2,5})'
    d['email'] = re.findall(email_query,d['mail_text'])
    if d['url']:
        d['has_url'] = 1
        d['num_url'] = len(d['url'])
        d['num_uniq_url'] = len(set(d['url']))
        d['num_url_repeats'] = d['num_url'] - d['num_uniq_url']
        d['url_len'] = []
        d['url_tld'] = []
        for i in d['url']:
            d['url_len'].append(len(i))
            d['url_tld'].append(
                get_tld(i, fix_protocol=True, fail_silently=True)
            )
            d['uniq_url_tld'] = set(d['url_tld'])
    else:
        d['url'] = ''
        d['has_url'] = 0
        d['num_url'] = 0
        d['num_uniq_url'] = 0
        d['url_len'] = 0
        d['url_tld'] = 0
        d['uniq_url_tld'] = 0
        d['num_url_repeats'] = 0
    if d['email']:
        d['has_email'] = 1
        d['num_email_addr'] = len(d['email'])
        d['num_uniq_email'] = len(set(d['email']))
    else:
        d['email'] = ''
        d['has_email'] = 0
        d['num_email_addr'] = 0
        d['num_uniq_email'] = 0
    soup = BeautifulSoup(d['mail_text'],'lxml')
    try:
        d['mail_text'] = soup.get_text().encode(
            'utf-8',
            'ignore'
        ).decode('unicode_escape').replace('\n',' ').replace('\t',' ')
    except:
        d['mail_text'] = soup.get_text().replace('\n',' ').replace('\t',' ')
    d['reply_to'] = mail.reply_to
    for k,v in mail.headers.items():
        d[k] = v
    d['body'] = mail.body
    d['text_plain'] = mail.text_plain
    if mail.attachments:
        d['has_attachments'] = 1
        d['num_attachments'] = len(mail.attachments)
        for i in mail.attachments:
            try:
                d['attachment_filename'].append(i['filename'])
            except:
                d['attachment_filename'] = []
                d['attachment_filename'].append(i['filename'])
            try:
                d['attachment_content_type'].append(i['mail_content_type'])
            except:
                d['attachment_content_type'] = []
                d['attachment_content_type'].append(i['mail_content_type'])
    else:
        d['has_attachments'] = 0
    if re.search(tryGetKeyValue(d, 'Return-Path'),tryGetKeyValue(d, 'From')):
        d['return_path_match_from'] = 1
    else:
        d['return_path_match_from'] = 0
    tld_match = re.match(
            '[^@]+@([^>]+)>',
            tryGetKeyValue(d, 'From')
        )
    if tld_match:
        d['from_tld'] = get_tld(
            tld_match.group(1),
            fix_protocol=True, 
            fail_silently=True
        )
    if 'content="text/html' in d['body'].lower():
        d['has_html_content'] = 1
    else:
        d['has_html_content'] = 0
    if 'script type="text/javascript' in d['body'].lower():
        d['has_javascript'] = 1
    else:
        d['has_javascript'] = 0
    if 'img src="cid:' in d['body'].lower():
        d['has_inline_img'] = 1
    else:
        d['has_inline_img'] = 0
    if 'Content-type' in d:
        d['Content-type'] = re.match('([^;]+);',d['Content-type']).group(1)
    else:
        d['Content-type'] = None
    if 'Date' in d:
        #d['DOTW'] = strftime('%a',strptime(d['Date'],'%a, %d %b %Y %H:%M:%S %z'))
        d['DOTW'] = strftime('%w',strptime(d['Date'],'%a, %d %b %Y %H:%M:%S %z'))
        d['HOTD'] = strftime('%H',strptime(d['Date'],'%a, %d %b %Y %H:%M:%S %z'))
    if mail.has_defects:
        d['has_defects'] = 1
    else:
        d['has_defects'] = 0
    return d

Build function to set the target value based on the directory

In [7]:
def get_target(d):
    if d == IGNORE_DIR:
        return 'ignore'
    elif d == INVESTIGATE_DIR:
        return 'investigate'

Build function to extract value only if the key exists

In [8]:
def tryGetKeyValue(d, key, return_value=''):
  """Attempts to return value of key from dictionary
  """
  try:
    return d[key]
  except:
    return return_value

Create dataframe, iterate through directories and add email features

In [9]:
df = pd.DataFrame()
for d,v in input_dirs.items():
    for f in input_dirs[d]:
        print(d, f)
        with open(os.path.join(d,f),'r',encoding='utf-8', errors='ignore') as raw_mail:
            mail_dict = get_email_text(raw_mail)
            df = df.append(
                {
                    #'filename': f,
                    'text': mail_dict['mail_text'],
                    'target': get_target(d),
                    'subject_len': mail_dict['subject_len'],
                    'body_len': mail_dict['body_len'],
                    'has_attachments': tryGetKeyValue(mail_dict, 'has_attachments',0),
                    'num_attachments': tryGetKeyValue(mail_dict, 'num_attachments',0),
                    #'attachment_filename': tryGetKeyValue(mail_dict, 'attachment_filename'),
                    #'attachment_content_type': tryGetKeyValue(mail_dict, 'attachment_content_type'),
                    'DKIM': tryGetKeyValue(mail_dict, 'X-BAEAI-DKIM'),
                    'DMARC': tryGetKeyValue(mail_dict, 'X-BAEAI-DMARC'),
                    'SPF': tryGetKeyValue(mail_dict, 'X-BAEAI-SPF'),
                    'return_path_match_from': mail_dict['return_path_match_from'],
                    'from_tld': tryGetKeyValue(mail_dict, 'from_tld'),
                    'Content-type': mail_dict['Content-type'],
                    'DOTW': tryGetKeyValue(mail_dict, 'DOTW'),
                    'HOTD': tryGetKeyValue(mail_dict, 'HOTD'),
                    #'url': mail_dict['url'],
                    'has_url': tryGetKeyValue(mail_dict, 'has_url',0),
                    'num_url': tryGetKeyValue(mail_dict, 'num_url',0),
                    'num_uniq_url': tryGetKeyValue(mail_dict, 'num_uniq_url',0),
                    #'email': tryGetKeyValue(mail_dict, 'email'),
                    'has_email': tryGetKeyValue(mail_dict, 'has_email',0),
                    'num_email_addr': tryGetKeyValue(mail_dict, 'num_email_addr',0),
                    'num_uniq_email': tryGetKeyValue(mail_dict, 'num_uniq_email',0),
                    'num_url_repeats': tryGetKeyValue(mail_dict, 'num_url_repeats',0),
                    #'url_len': mail_dict['url_len'],
                    #'url_tld': mail_dict['url_tld'],
                    #'uniq_url_tld': mail_dict['uniq_url_tld'],
                    'has_html_content': tryGetKeyValue(mail_dict, 'has_html_content',0),
                    'has_javascript': tryGetKeyValue(mail_dict, 'has_javascript',0),
                    'has_inline_img': tryGetKeyValue(mail_dict, 'has_inline_img',0),
                    'TAP-Score': tryGetKeyValue(mail_dict, 'X-USANET-TAP-Score',-1),
                    #'links': tryGetKeyValue(mail_dict, 'links'),
                    'num_link': tryGetKeyValue(mail_dict, 'num_link',0),
                    'num_uniq_link': tryGetKeyValue(mail_dict, 'num_uniq_link',0),
                    'has_repeat_link': tryGetKeyValue(mail_dict, 'has_repeat_link',0),
                    #'masq_link': tryGetKeyValue(mail_dict, 'masq_link'),
                    'has_masq_link': tryGetKeyValue(mail_dict, 'has_masq_link',0),
                    'num_masq_link': tryGetKeyValue(mail_dict, 'num_masq_link',0),
                    #'masq_link_tld': tryGetKeyValue(mail_dict, 'masq_link_tld'),
                    'is_multipart': tryGetKeyValue(mail_dict, 'is_mulitpart', 0),
                    'has_defects': mail_dict['has_defects'],
                    'num_email_link': tryGetKeyValue(mail_dict, 'num_email_link',0),
                    'has_unsubscribe_link': tryGetKeyValue(mail_dict, 'has_unsubscribe_link', 0),
                }, 
                ignore_index=True
            )

/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Goodchem Document.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Validate your Office 365 account.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- INVOICE YAX-80-97157 Tom Jensen.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice Problem 3.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice from Karen Laman.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Case-637290015585-228.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- 7 Past Due Invoices.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- lmportant Message.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- [Spam] Erinnerung an die Rechnungszahlung.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Re-[1].eml
/User

/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice number 22028027-LFYJ#WDBB-2017 (27 Nov 17) Notification.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Inbound- 5 new.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Past due invoice .eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice # 530522431 Problem.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- UPS Ship Notification, Tracking Number 2U51525393363758.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Newly signed Project invitation - please view proposal-.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Your document is signed.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Fax Received- CSID- K7V 6B3 1910 CID- 90643249844.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Preview S

/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice[2].eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- thy.nguyen@pinnacol.com Be careful! malicious software ID a5iQmYe1e.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Your new Chase payment notice.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- PAYMENT EAA-90-22623  .eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- ICBC Payment Remittance.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- File shared 11-21-2017 on behalf of Kim Willoughby.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Wrong Invoice   Address.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Scanned Document - (Past due Invoice-083821.Pdf & Contract_Agreements.PDF).eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- [SPAM] New pay

/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Payment notice for invoice #1647302.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Recent payment notification to Carol Rusten.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- UPS Express Domestic.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Yоu hаvе thе tаlеnt tо роsе fоr thе саmеrа, dоn't bе nеrvоus..eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Coseco Invoice IR1-52583.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- IMPORTANT MAIL.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Pay Invoice.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice #9066829 from Lori Whitesides.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- [Spam] IMPORTANT - William Olberding  Invitation to view..eml
/User

/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Coloradohandandarm ( Teusday 27, 2018).eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Dylan Grant.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Delivery Status Notification.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Invoice PUJ #73722-AUL#YSML-2017 (21 Sep 17).eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- 66925_whitepaperfivemusthavesinacloudfinancialmanagementsystemus.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Question.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Important Document 2.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Email Verification.eml
/Users/worshamn/Documents/emailProject/InvestigateFix [Phish Alert] FW- Your Invoice Is Attached - 0000125794.eml
/Users/worshamn/Documents/emailProject/

...

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: DeprecationWarning: invalid escape sequence '\H'


/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- {rp}, you are selected..eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- We just launched our rewards program! Save $5 or more today..eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Gartner Magic Quadrant for P&C Core Platforms in North America.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Not Happy with Your Current CRM System.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- GDPR consent- Please confirm your subscription with dacadoo.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- They put a spy cam in a USB charger.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- ILP Conf Agenda_111617WO names.docx.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- How to Adopt SAP on AWS Easily.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- HR's Guide To 

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:96: DeprecationWarning: invalid escape sequence '\#'


/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- You have a new encrypted message from mlloyd@springersteinberg.com.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Credit Score.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- IR Filing Alert - Quarterly report with a continuing view of a company's financial position - Form 10-Q.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Delivery Status Notification (Failure).eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- [Spam] Mike, I have 10 ideas I think you'll be interested in..eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- please add me to your list.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- The V.I.P. Event Starts TODAY.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- NOVAtime  Territory Takeover  .eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Al

/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- You're Invited!.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Fwd-.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Saturday Matinee.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Lunch - 9-19-17 - Learn about Insider Threat with InteliSecure and Forcepoint .eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Upcoming Commvault® Cloud Data Protection Customer Survey.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- All Shark-Tank judges agree this is the best.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- NFL's Ticket Manager Application.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Watch the latest recommendations from your network.eml
/Users/worshamn/Documents/emailProject/IgnoreFix [Phish Alert] FW- Social Web Investigations & Claims Support Services.eml
/User

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 34 columns):
Content-type              773 non-null object
DKIM                      784 non-null object
DMARC                     784 non-null object
DOTW                      784 non-null object
HOTD                      784 non-null object
SPF                       784 non-null object
TAP-Score                 784 non-null object
body_len                  784 non-null float64
from_tld                  784 non-null object
has_attachments           784 non-null float64
has_defects               784 non-null float64
has_email                 784 non-null float64
has_html_content          784 non-null float64
has_inline_img            784 non-null float64
has_javascript            784 non-null float64
has_masq_link             784 non-null float64
has_repeat_link           784 non-null float64
has_unsubscribe_link      784 non-null float64
has_url                   784 non-null float64
is_multipa

Create a function to clean the text

In [11]:
def clean_text(
    docs, 
    remove_urls=True,
    remove_emails=True,
    lemmatize=True,
    remove_stopwords=True, 
    custom_stopwords=None,
#     term_min_len=0,
):
    #only use parts of spaCy needed
    nlp = spacy.load('en', disable=['parser','ner','textcat'])
    #remove urls
    if remove_urls:
        print('remove URLS')
        docs = [
            re.sub('(?i)(?:www|https?)(?:://)?[^\s]+','',text)
            for text in docs
        ]
    #remove emails
    if remove_emails:
        print('remove email addresses')
        docs = [
            re.sub('(?i)[\w.]+@[\w.]+\.[\w.]{2,5}','',text)
            for text in docs
        ]
    #remove punct and digits
    print('removing punctuation and digits and change to lowercase')
    table = str.maketrans({key: None for key in string.punctuation + string.digits})
    clean_docs = [
        str(d).lower().translate(table)
        for d in docs
    ]
        
    #tokenize in spacy
    if lemmatize:
        print('spacy tokenization')
        nlp_docs = [nlp(d) for d in clean_docs]
        #lemmatization, words like I get changed into -PRON- so leave them alone
        if remove_stopwords:
            print('lemmatization and remove stopwords')
            if custom_stopwords:
                custom_stopwords = set(custom_stopwords)
            else:
                custom_stopwords = []
            lemmatized_docs = [
                [
                    w.lemma_ 
                    for w in d
                    if (w.lemma_ != '-PRON-' and not w.is_stop and w.lemma_ not in custom_stopwords) 
                ]
                for d in nlp_docs
            ]
        else:
            print('lemmatization')
            lemmatized_docs = [
            [
                w.lemma_
                if w.lemma_ != '-PRON-'
                else w.lower_
                for w in d
                #if (w.lemma_ != '-PRON-' and len(w.lemma_)>term_min_len)
            ]
            for d in nlp_docs
        ]
    if lemmatized_docs:
        clean_docs = lemmatized_docs
    
    # join tokens back into doc
    clean_docs = [
        ' '.join(l) 
        for l in clean_docs
    ]

    return clean_docs

Clean the text

In [12]:
from time import time
t0 = time()
corpus = clean_text(
    df['text'], 
)
print("done in %0.3fs" % (time() - t0))

remove URLS
remove email addresses
removing punctuation and digits and change to lowercase
spacy tokenization
lemmatization and remove stopwords
done in 9.115s


In [13]:
df['cleaned_text'] = pd.Series(corpus).values

Convert cells that are not recognized as a number or just to get rid of float

In [14]:
#https://stackoverflow.com/a/28910914
for col in [
    'body_len',
    'has_attachments',
    'has_defects',
    'has_email',
    'has_html_content',
    'has_inline_img',
    'has_javascript',
    'has_masq_link',
    'has_repeat_link',
    'has_unsubscribe_link',
    'has_url',
    'is_multipart',
    'num_attachments',
    'num_email_addr',
    'num_email_link',
    'num_link',
    'num_masq_link',
    'num_uniq_email',
    'num_uniq_link',
    'num_uniq_url',
    'num_url',
    'num_url_repeats',
    'return_path_match_from',
    'subject_len',
    #'TAP-Score',
]:
    df[col] = df[col].astype(int)
    

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 35 columns):
Content-type              773 non-null object
DKIM                      784 non-null object
DMARC                     784 non-null object
DOTW                      784 non-null object
HOTD                      784 non-null object
SPF                       784 non-null object
TAP-Score                 784 non-null object
body_len                  784 non-null int64
from_tld                  784 non-null object
has_attachments           784 non-null int64
has_defects               784 non-null int64
has_email                 784 non-null int64
has_html_content          784 non-null int64
has_inline_img            784 non-null int64
has_javascript            784 non-null int64
has_masq_link             784 non-null int64
has_repeat_link           784 non-null int64
has_unsubscribe_link      784 non-null int64
has_url                   784 non-null int64
is_multipart              784 no

Break the continous data into ranges for binary conversion

In [16]:
#https://stackoverflow.com/a/40548606
df['body_len'] = pd.qcut(df['body_len'],20,duplicates='drop')
df['subject_len'] = pd.qcut(df['subject_len'],10,duplicates='drop')
df['num_attachments'] = pd.cut(df['num_attachments'],5)
df['num_email_addr'] = pd.cut(df['num_email_addr'],5)
df['num_email_link'] = pd.cut(df['num_email_link'],5)
df['num_link'] = pd.qcut(df['num_link'],10,duplicates='drop')
df['num_masq_link'] = pd.cut(df['num_masq_link'],3)
df['num_uniq_email'] = pd.qcut(df['num_uniq_email'],10,duplicates='drop')
df['num_uniq_url'] = pd.qcut(df['num_uniq_url'],10,duplicates='drop')
df['num_url'] = pd.qcut(df['num_url'],10,duplicates='drop')
df['num_url_repeats'] = pd.qcut(df['num_url_repeats'],10,duplicates='drop')

Change the continuous and categorical data into one-hot encoding (binary data)

In [17]:
categorical_cols = [
    'DKIM',
    'DMARC',
    'SPF',
    'from_tld',
    'Content-type',
    'DOTW',
    'HOTD',
    'TAP-Score',
    'body_len',
    'subject_len',
    'num_attachments',
    'num_email_addr',
    'num_email_link',
    'num_link',
    'num_masq_link',
    'num_uniq_email',
    'num_uniq_url',
    'num_url',
    'num_url_repeats',
]

In [18]:
df_categorical = pd.get_dummies(df[categorical_cols])

In [19]:
df.drop(categorical_cols, axis=1, inplace=True)

In [20]:
df = pd.concat([df,df_categorical], axis=1)

In [21]:
pd.set_option('display.max_colwidth', 40)

In [22]:
df.head()

,has_attachments,has_defects,has_email,has_html_content,has_inline_img,has_javascript,has_masq_link,has_repeat_link,has_unsubscribe_link,has_url,...,"num_url_(-0.001, 1.0]","num_url_(1.0, 2.0]","num_url_(2.0, 4.0]","num_url_(4.0, 6.0]","num_url_(6.0, 11.0]","num_url_(11.0, 17.0]","num_url_(17.0, 341.0]","num_url_repeats_(-0.001, 1.0]","num_url_repeats_(1.0, 3.0]","num_url_repeats_(3.0, 95.0]"
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [23]:
for i in df.columns:
    print(i)

has_attachments
has_defects
has_email
has_html_content
has_inline_img
has_javascript
has_masq_link
has_repeat_link
has_unsubscribe_link
has_url
is_multipart
num_uniq_link
return_path_match_from
target
text
cleaned_text
DKIM_
DKIM_FAIL
DKIM_NONE
DKIM_PASS
DMARC_
DMARC_absent
DMARC_none
DMARC_pass
DMARC_quarantine
DMARC_reject
SPF_
SPF_ERROR
SPF_NEUTRAL
SPF_NONE
SPF_PASS
SPF_SOFTFAIL
from_tld_
from_tld_ac.uk
from_tld_bg
from_tld_bid
from_tld_biz
from_tld_ca
from_tld_ch
from_tld_co
from_tld_co.jp
from_tld_co.uk
from_tld_co.za
from_tld_com
from_tld_com.ar
from_tld_com.br
from_tld_com.do
from_tld_com.ec
from_tld_com.mx
from_tld_com.my
from_tld_com.pk
from_tld_de
from_tld_edu
from_tld_email
from_tld_es
from_tld_eu
from_tld_fi
from_tld_fr
from_tld_gob.cl
from_tld_gob.ec
from_tld_gov
from_tld_gr
from_tld_guru
from_tld_hu
from_tld_ie
from_tld_info
from_tld_io
from_tld_it
from_tld_k12.in.us
from_tld_k12.wi.us
from_tld_mx
from_tld_ne.jp
from_tld_net
from_tld_net.pl
from_tld_net.tw
from_tld_org
fr

Split the sample set into test and training sets

In [24]:
df_target = df['target']
df_feats = df.drop(['target','text'],axis=1)
train_feats, test_feats, train_labels, test_labels = train_test_split(
    df_feats, 
    df_target, 
    test_size=0.20, 
    random_state=7350
)

In [25]:
train_feats.shape

(627, 201)

In [26]:
test_feats.shape

(157, 201)

In [27]:
train_labels.value_counts()

ignore         318
investigate    309
Name: target, dtype: int64

#### TFIDF Creation

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
t0 = time()
vectorizer = TfidfVectorizer(
    ngram_range=(1,3),
    #max_df=0.9, 
    min_df=2,
    use_idf=True
)
train_feats_text = vectorizer.fit_transform(train_feats['cleaned_text'].values.tolist())
test_feats_text = vectorizer.transform(test_feats['cleaned_text'].values.tolist()) 
print("done in %0.3fs" % (time() - t0))

done in 0.474s


In [29]:
train_feats_text.shape

(627, 8943)

In [30]:
test_feats_text.shape

(157, 8943)

Change TFIDF back to dataframe so it can be concatinated with other features

In [31]:
#https://stackoverflow.com/a/50624143
train_feats_text_df = pd.DataFrame(train_feats_text.toarray(), columns=vectorizer.get_feature_names())

In [32]:
train_feats_text_df.head()

,aaa,ab,abbuchung,abend,ability,able,able access,able hold,able hold phone,absent,...,youthful beauty,youtube,youtube google,youtube instagram,yu,yð¾u,zip,zip code,zipongo,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
train_feats = pd.concat([
    train_feats.reset_index(drop=True),
    train_feats_text_df.reset_index(drop=True)
],axis=1)
train_feats.drop('cleaned_text',axis=1,inplace=True)
train_feats.shape

(627, 9143)

Do the same for the test features

In [34]:
test_feats_text_df = pd.DataFrame(test_feats_text.toarray(), columns=vectorizer.get_feature_names())
test_feats_text_df = test_feats_text_df.add_prefix('LSA_')
test_feats = pd.concat([
    test_feats.reset_index(drop=True),
    test_feats_text_df.reset_index(drop=True)
],axis=1)
test_feats.drop('cleaned_text',axis=1,inplace=True)
test_feats.shape

(157, 9143)

Function to keep track of scoring and for printing results out

In [35]:
score_dict = {}
def scoring(score_dict,train_feats,train_labels,test_feats,test_labels,clf):
    if 'accuracy' not in score_dict:
        score_dict['accuracy'] = []
    if 'f1' not in score_dict:
        score_dict['f1'] = []
    if 'recall' not in score_dict:
        score_dict['recall'] = []    
    if 'FN' not in score_dict:
        score_dict['FN'] = []
    clf_name = re.findall('(^[^\(]+)\(',str(clf))[0]
    already_seen = clf.score(train_feats, train_labels)
    accuracy = clf.score(test_feats, test_labels)
    pred = clf.predict(test_feats)
    f1 = f1_score(test_labels, pred, pos_label='investigate')
    recall = recall_score(test_labels, pred, pos_label='investigate')
    cnf_matrix = confusion_matrix(test_labels, pred)
    FN = cnf_matrix[1][0]
    false_negative = cnf_matrix[1][0]
    score_dict['accuracy'].append((clf_name,accuracy))
    score_dict['f1'].append((clf_name,f1))
    score_dict['recall'].append((clf_name,recall))
    score_dict['FN'].append((clf_name,false_negative))
    print(clf_name + ' Scores:\n')
    print('Accuracy of data already seen: %0.4f' % already_seen)
    print('Accuracy of data not seen: %0.4f' % accuracy)
    print('F1 score: %0.4f' % f1)
    print('Recall score: %0f' % recall)
    print('False Negatives: %0d' % FN)
    return score_dict

#### LinearSVM Gridsearch

In [41]:
#https://github.com/scikit-learn/scikit-learn/issues/8918
#http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html#grid-search
from sklearn.svm import LinearSVC 
t0 = time()
svm_clf = LinearSVC()
parameters = {
    'C': np.append(np.logspace(-6, -1, 5),[1,5,10]),
    'tol': np.logspace(-10, -1, 5),
#     'penalty': ('l1', 'l2'),
    'loss': ('hinge', 'squared_hinge'),
    'fit_intercept': (True, False),
    'max_iter': (1000, 2000)
}
gs_svm_clf = GridSearchCV(estimator=svm_clf, param_grid=parameters, n_jobs=-1,cv=5,return_train_score=False)
gs_svm_clf.fit(train_feats, train_labels)
print("done in %0.3fs" % (time() - t0))
print(gs_svm_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_svm_clf.best_params_[param_name]))

done in 179.287s
0.9011164274322169
C: 5.0
fit_intercept: True
loss: 'hinge'
max_iter: 1000
tol: 0.0005623413251903491


#### Try with scaling the data first

In [42]:
# #https://stackoverflow.com/a/37199623
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
train_feats = min_max_scaler.fit_transform(train_feats)

In [43]:
#https://github.com/scikit-learn/scikit-learn/issues/8918
#http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html#grid-search
from sklearn.svm import LinearSVC 
t0 = time()
svm_clf = LinearSVC()
parameters = {
    'C': np.append(np.logspace(-6, -1, 5),[1,5,10]),
    'tol': np.logspace(-10, -1, 5),
#     'penalty': ('l1', 'l2'),
    'loss': ('hinge', 'squared_hinge'),
    'fit_intercept': (True, False),
    'max_iter': (1000, 2000)
}
gs_svm_clf = GridSearchCV(estimator=svm_clf, param_grid=parameters, n_jobs=-1,cv=5,return_train_score=False)
gs_svm_clf.fit(train_feats, train_labels)
print("done in %0.3fs" % (time() - t0))
print(gs_svm_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_svm_clf.best_params_[param_name]))

done in 169.909s
0.9330143540669856
C: 0.1
fit_intercept: True
loss: 'hinge'
max_iter: 1000
tol: 1e-10
